In [27]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import random
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from keras.layers import Dropout
from keras import regularizers
from matplotlib.pyplot import rcParams
%matplotlib inline
rcParams['figure.figsize'] = 10,8
sns.set(style='whitegrid', palette='muted',
        rc={'figure.figsize': (15,10)})
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

from numpy.random import seed
from tensorflow import set_random_seed

In [28]:
train_dataset = pd.read_csv("energy_train.csv")
test_dataset = pd.read_csv("energy_test.csv")

train_dataset = train_dataset.set_index("id")
test_dataset = test_dataset.set_index("id")
train_dataset.dropna(axis=0,  inplace=True)


train_dataset.columns = ['Date','WattHour', 'Lumin','D1','M1','D2','M2','D3','M3','D4','M4','D5','M5','D6','M6','D7','M7','D8','M8','D9','M9','D_Cout','Pressure','M_Cout','Wind','Clarity','Dew','R1','R2']

test_dataset.columns = ['Date', 'Lumin','D1','M1','D2','M2','D3','M3','D4','M4','D5','M5','D6','M6','D7','M7','D8','M8','D9','M9','D_Cout','Pressure','M_Cout','Wind','Clarity','Dew','R1','R2']



train_dataset.dropna(axis=0,inplace=True)
train_dataset['Lumin'] = train_dataset['Lumin'].astype(float)
test_dataset['Lumin'] = test_dataset['Lumin'].astype(float)
train_dataset.round(decimals=2)
test_dataset.round(decimals=2)
np.nan_to_num(train_dataset)
np.nan_to_num(test_dataset)
train_dataset.fillna("bfill")
test_dataset.fillna("bfill")
train_dataset

,Date,WattHour,Lumin,D1,M1,D2,M2,D3,M3,D4,...,D9,M9,D_Cout,Pressure,M_Cout,Wind,Clarity,Dew,R1,R2
id,,,,,,,,,,,,,,,,,,,,,
1,2016-01-11 17:00:00,60,30.0,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,...,17.033333,45.530000,6.600000,733.500000,92.000000,7.000000,63.000000,5.300000,13.275433,13.275433
2,2016-01-11 17:10:00,60,30.0,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,...,17.066667,45.560000,6.483333,733.600000,92.000000,6.666667,59.166667,5.200000,18.606195,18.606195
3,2016-01-11 17:20:00,50,30.0,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,...,17.000000,45.500000,6.366667,733.700000,92.000000,6.333333,55.333333,5.100000,28.642668,28.642668
4,2016-01-11 17:30:00,50,40.0,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,...,17.000000,45.400000,6.250000,733.800000,92.000000,6.000000,51.500000,5.000000,45.410389,45.410389
7,2016-01-11 18:00:00,60,50.0,19.890000,45.766667,19.200000,44.500000,19.790000,44.900000,18.890000,...,17.000000,45.290000,5.900000,734.100000,92.000000,5.000000,40.000000,4.700000,47.233763,47.233763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14990,2016-04-24 19:10:00,90,0.0,22.100000,35.845000,19.790000,37.060000,22.500000,33.790000,21.614286,...,20.390000,33.933333,7.133333,757.633333,65.333333,5.000000,40.000000,1.016667,21.838660,21.838660
14991,2016-04-24 19:20:00,70,0.0,22.100000,35.730000,19.790000,37.200000,22.500000,33.790000,21.600000,...,20.390000,33.723333,6.766667,757.666667,66.666667,5.000000,40.000000,0.933333,18.788502,18.788502
14994,2016-04-24 19:50:00,120,0.0,22.100000,35.900000,19.700000,37.500000,22.390000,33.966667,21.542857,...,20.290000,33.290000,5.666667,757.766667,70.666667,5.000000,40.000000,0.683333,27.295133,27.295133


In [29]:
X = train_dataset.drop(['WattHour','Date'], axis=1)
y = train_dataset["WattHour"]

In [30]:

from sklearn.model_selection import train_test_split
# here I used 20% data for test_data and 80% as train_data
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [31]:
model = Sequential([
    Dense(32, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [32]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [33]:
hist = model.fit(np.array(x_train), np.array(y_train),
          batch_size=32, epochs=100)

Epoch 1/100
5194/5194 [==============================] - 0s 41us/step - loss: nan - accuracy: 0.0000e+00
Epoch 2/100
5194/5194 [==============================] - 0s 26us/step - loss: nan - accuracy: 0.0000e+00
Epoch 3/100
5194/5194 [==============================] - 0s 26us/step - loss: nan - accuracy: 0.0000e+00
Epoch 4/100
5194/5194 [==============================] - 0s 27us/step - loss: nan - accuracy: 0.0000e+00
Epoch 5/100
5194/5194 [==============================] - 0s 25us/step - loss: nan - accuracy: 0.0000e+00
Epoch 6/100
5194/5194 [==============================] - 0s 28us/step - loss: nan - accuracy: 0.0000e+00
Epoch 7/100
5194/5194 [==============================] - 0s 27us/step - loss: nan - accuracy: 0.0000e+00
Epoch 8/100
5194/5194 [==============================] - 0s 26us/step - loss: nan - accuracy: 0.0000e+00
Epoch 9/100
5194/5194 [==============================] - 0s 25us/step - loss: nan - accuracy: 0.0000e+00
Epoch 10/100
5194/5194 [==============================]

5194/5194 [==============================] - 0s 25us/step - loss: nan - accuracy: 0.0000e+00
Epoch 78/100
5194/5194 [==============================] - 0s 25us/step - loss: nan - accuracy: 0.0000e+00
Epoch 79/100
5194/5194 [==============================] - 0s 26us/step - loss: nan - accuracy: 0.0000e+00
Epoch 80/100
5194/5194 [==============================] - 0s 24us/step - loss: nan - accuracy: 0.0000e+00
Epoch 81/100
5194/5194 [==============================] - 0s 24us/step - loss: nan - accuracy: 0.0000e+00
Epoch 82/100
5194/5194 [==============================] - 0s 24us/step - loss: nan - accuracy: 0.0000e+00
Epoch 83/100
5194/5194 [==============================] - 0s 25us/step - loss: nan - accuracy: 0.0000e+00
Epoch 84/100
5194/5194 [==============================] - 0s 25us/step - loss: nan - accuracy: 0.0000e+00
Epoch 85/100
5194/5194 [==============================] - 0s 25us/step - loss: nan - accuracy: 0.0000e+00
Epoch 86/100
5194/5194 [==============================] - 0

In [34]:
model.evaluate(x_test, y_test)[1]

1299/1299 [==============================] - 0s 31us/step


0.0

In [35]:
model_3 = Sequential([
    Dense(1000, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.3),
    Dense(1000, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.3),
    Dense(1000, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.3),
    Dense(1000, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.3),
    Dense(1, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01))
])

In [36]:
model_3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [39]:
hist_3 = model_3.fit(np.array(x_train), np.array(y_train),
          batch_size=32, epochs=100)

Epoch 1/100
5194/5194 [==============================] - 9s 2ms/step - loss: -9026927137271.4414 - accuracy: 0.0000e+00
Epoch 2/100
5194/5194 [==============================] - 9s 2ms/step - loss: -1529610894531071.7500 - accuracy: 0.0000e+00
Epoch 3/100
5194/5194 [==============================] - 9s 2ms/step - loss: -24926112011835212.0000 - accuracy: 0.0000e+00
Epoch 4/100
5194/5194 [==============================] - 9s 2ms/step - loss: -153706139413183072.0000 - accuracy: 0.0000e+00
Epoch 5/100
5194/5194 [==============================] - 9s 2ms/step - loss: -564189676332008448.0000 - accuracy: 0.0000e+00
Epoch 6/100
5194/5194 [==============================] - 9s 2ms/step - loss: -1539021535467387392.0000 - accuracy: 0.0000e+00
Epoch 7/100
5194/5194 [==============================] - 9s 2ms/step - loss: -3474386156353452032.0000 - accuracy: 0.0000e+00
Epoch 8/100
5194/5194 [==============================] - 9s 2ms/step - loss: -6859362193559340032.0000 - accuracy: 0.0000e+00
Epoch

5194/5194 [==============================] - 8s 2ms/step - loss: -46553311810516735229952.0000 - accuracy: 0.0000e+00
Epoch 64/100
5194/5194 [==============================] - 8s 2ms/step - loss: -49645103618040275140608.0000 - accuracy: 0.0000e+00
Epoch 65/100
5194/5194 [==============================] - 8s 2ms/step - loss: -52922416271606748807168.0000 - accuracy: 0.0000e+00
Epoch 66/100
5194/5194 [==============================] - 8s 2ms/step - loss: -56237645148367326543872.0000 - accuracy: 0.0000e+00
Epoch 67/100
5194/5194 [==============================] - 8s 2ms/step - loss: -59698180067607187554304.0000 - accuracy: 0.0000e+00
Epoch 68/100
5194/5194 [==============================] - 9s 2ms/step - loss: -63460095915530400038912.0000 - accuracy: 0.0000e+00
Epoch 69/100
5194/5194 [==============================] - 9s 2ms/step - loss: -67295992991081625026560.0000 - accuracy: 0.0000e+00
Epoch 70/100
5194/5194 [==============================] - 10s 2ms/step - loss: nan - accuracy: 0